In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
%matplotlib notebook

import os
import glob
import math
import pandas as pd
import numpy as np
import itertools
import random

import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.patches as patches

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Layer, Conv2D, BatchNormalization, Concatenate, UpSampling2D, MaxPool2D, Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from keras.utils import np_utils
from keras.utils.vis_utils import model_to_dot
from tensorflow.keras.callbacks import EarlyStopping

import tensorflow.keras.backend as K

from IPython.display import SVG

import tensorflow_probability as tfp


from sklearn.metrics import classification_report, confusion_matrix
from sklearn import preprocessing
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer, normalize
from sklearn.preprocessing import MinMaxScaler, Normalizer
import warnings

warnings.filterwarnings('ignore')

from ipywidgets import FloatProgress
from IPython.display import display

fig_width = 12
plt.rcParams["font.size"] = 40
plt.rcParams['axes.labelsize'] = 40
plt.rcParams['axes.labelweight'] = 'bold'
#plt.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
#plt.rcParams.keys()

from codes.utils.HAR_utils import get_activity_index, plot_confusion_matrix, get_Train_Test, timeseries_standardize
from codes.models.attention import Attention2D

>>> Done:
>>> Done: Succesfully imported Utils module
>>> Done: Succesfully imported Utils module
Added attention files


## Transform Data

In [3]:
def split_every_n_rows(dataframe, chunk_size=10000):
    chunks = []
    num_chunks = len(dataframe) // chunk_size + 1

    for index in range(num_chunks):
        chunks.append(dataframe[index * chunk_size:(index+1) * chunk_size])

    return chunks

def create_x_y_t(data, features, reqlen=10000, dummy_lval=10):
    t_X = []
    t_Y = []
    t_times = []
    normalizer = Normalizer()

    for i in range(len(data)):
        if len(data[i]) < reqlen:
            x = np.array(data[i][features].values)
            xz = np.zeros((reqlen-len(x),len(features)))
            xs = normalizer.fit_transform(x)
            sx = np.concatenate((xs,xz))

            y = np.array(data[i]['l_val'].values)
            z = np.array([dummy_lval]*(reqlen-len(y)))
            sy = np.concatenate((y,z))
    
            t = np.array(data[i]['timestamp'].values)
            st = np.concatenate((t,z))
    
            t_X.append([sx])
            t_Y.append([sy])
            t_times.append(st)

        else:
            x = np.array(data[i][features].values)
            xs = normalizer.fit_transform(x)

            y = np.array(data[i]['l_val'].values)

            t = np.array(data[i]['timestamp'].values)

            t_X.append([xs])
            t_Y.append([y])
            t_times.append(t)
    t_X = np.array(t_X)
    t_Y = np.array(t_Y)
    #print('t_X.shape',t_X.shape)

    return t_X, t_Y, t_times

In [4]:
def get_train_test_bird_dev(in_df, trn='A', reqlen=10000, shuffle=False):
    in_df.dropna(inplace=True)
    birds = list(in_df.drop_duplicates(subset=['animal_tag'],keep = 'first')['animal_tag'])
    features = in_df.columns[2:-2]
    print('features: ', features)

    if trn == 'A':
        selector = random.choice(birds)
    else:
        selector = trn

    df_d = {}
    for bird in birds:
        bird_df = in_df[in_df['animal_tag'] == bird]
        bird_df_l = split_every_n_rows(bird_df, reqlen)
        df_d.update({bird:bird_df_l})
        print(bird)

    print(selector)
    tr_birds = birds
    tr_birds.remove(selector)

    test_data = df_d[selector]
    testX, testY, testtimes = create_x_y_t(test_data, features, reqlen=reqlen, dummy_lval=10)
    testX = np.array(testX)
    testY = np.array(testY)
    times = np.array(testtimes)

    trX = []
    trY = []
    for bird in tr_birds:
        train_data = df_d[bird]
        t_X , t_Y, t_times = create_x_y_t(train_data, features, reqlen=reqlen, dummy_lval=10)
        #t_X = np.array(t_X)
        #t_Y = np.array(t_Y)
        trX.append(t_X)
        trY.append(t_Y)
    trX = np.vstack(trX)
    trY = np.vstack(trY)

    if shuffle:
        p = list(zip(trX,trY))
        random.shuffle(p)
        trX, trY = zip(*p)
        trX = list(trX)
        trY = list(trY)

    trainX = np.array(trX)
    trainY = np.array(trY)

    print('train X(shape): ', trainX.shape)
    print('train Y(shape): ', trainY.shape)
    print('test X(shape): ', testX.shape)
    print('test Y(shape): ', testY.shape)
    print('times(shape): ',times.shape)

    return trainX, testX, trainY, testY, times, selector



In [5]:
def get_train_test_bird(in_df,shuffle=False,trn='A'):
    
    in_df.dropna(inplace=True)
    birds = list(in_df.drop_duplicates(subset=['animal_tag'],keep = 'first')['animal_tag'])
    max_len = 0

    for bird in birds:
        c_len = len(in_df[in_df['animal_tag'] == bird])
        
        if c_len > max_len:
            print(bird,' : ',c_len,' > ',max_len,' ? ',c_len > max_len)
            max_len = c_len
        print('bird: ',bird,'lenght: ',c_len)
    reqlen = int((int(max_len/512)+1)*512)
    print('reqlen: ',reqlen)

    features = in_df.columns[2:-2]
    X = []
    Y = []
    times = []
    normalizer = Normalizer()

    for bird in birds:
            x = np.array(in_df[in_df['animal_tag'] == bird][features].values)
            #print('x.shape',x.shape)
            xz = np.zeros((reqlen-len(x),len(features)))
            xs = normalizer.fit_transform(x)
            sx = np.concatenate((x,xz))
    
            y = np.array(in_df[in_df['animal_tag'] == bird]['l_val'].values)
            z = np.array([10]*(reqlen-len(y)))
            sy = np.concatenate((y,z))
    
            t = np.array(in_df[in_df['animal_tag'] == bird]['timestamp'].values)
            st = np.concatenate((t,z))
    
            X.append([sx])
            Y.append([sy])
            times.append(st)

    if trn != 1000:
        i=0
        if shuffle == True:
            shuffle == False
            print("Warning: shuffle cannot be used when selecting a specific bird")
        for bird in birds:
            if bird == trn:
                selector = i
            i=1+i
    else:
        selector = random.randint(0,len(birds)-1)

    if shuffle:
        p = list(zip(X,Y))
        random.shuffle(p)
        X, Y = zip(*p)
        X = list(X)
        Y = list(Y)

    X = np.array(X)
    Y = np.array(Y)

    times = np.array(times)
    print('X(shape): ',X.shape)
    print('Y(shape): ',Y.shape)
    print('times(shape): ',times.shape)

    testX = np.array([X[selector]])
    testY = np.array([Y[selector]])
    testtimes=times[selector]
    trainX = np.delete(X,selector,0)
    trainY = np.delete(Y,selector,0)
    
    return trainX,testX,trainY,testY,testtimes,selector

# Network design

In [6]:
def down_block(x, filters, kernel_size=(1, 3), padding="same", strides=1, name="down_"):
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu",name=name+"1")(x)
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu",name=name+"2")(c)
    p = MaxPool2D((1, 2),name=name+"M")(c)
    return c, p

def up_block(x, skip, filters, kernel_size=(1, 3), padding="same", strides=1, name="up_"):
    us = UpSampling2D((1, 2),name=name+"U")(x)
    concat = Concatenate()([us, skip])
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu",name=name+"1")(concat)
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu",name=name+"2")(c)
    return c

def bottleneck(x, filters, kernel_size=(1, 3), padding="same", strides=1):
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu",name="btl1")(x)
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu",name="btl2")(c)
    c = Dropout(0.5)(c)
    return c

def build_unet_model(height,channels,class_num,f=32):
    
    inputs = Input((1, height, channels))
    
    p0 = inputs
    c1, p1 = down_block(p0, f, name="down_1_")
    c2, p2 = down_block(p1, f*2, name="down_2_")
    c3, p3 = down_block(p2, f*4, name="down_3_")
    p3 = Dropout(0.5)(p3)
    
    bn = bottleneck(p3, f*8)
    
    u1 = up_block(bn, c3, f*4, name="up_1_")
    u2 = up_block(u1, c2, f*2, name="up_2_")
    u3 = up_block(u2, c1, f, name="up_3_")
    
    outputs = Dense(class_num, activation = 'softmax')(u3)
    model = Model(inputs, outputs)
    
    return model

def ApplyUnet_dev(trainX, trainY, testX, testY,TestTimes, activities, nb_epoch=100, fi=32):
    
    target_names,indices = get_activity_index(activities)
    #print(target_names)
    indices.append(len(target_names))
    target_names.append('EPadding')
    
    trainY_b = np_utils.to_categorical(trainY, len(target_names))
    testY_b = np_utils.to_categorical(testY, len(target_names))
    lb = LabelBinarizer()
    
    print(indices)
    lb.fit(indices)
    
    model = build_unet_model(trainX.shape[2],trainX.shape[3],len(target_names),f=fi)
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["acc"])
    model.summary()
    
    model_checkpoint = keras.callbacks.ModelCheckpoint('bioattunet-2022_membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)
    history = model.fit(trainX, trainY_b, epochs=nb_epoch, validation_split=0.2,callbacks=[model_checkpoint], verbose=0)
    
    fig, (tr_plt,cm_plt) = plt.subplots(nrows=2, ncols=1, figsize=(6, 6))
    
    tr_plt.plot(history.history['loss'])
    tr_plt.plot(history.history['val_loss'])
    tr_plt.set_title('Model loss')
    tr_plt.set_ylabel('Loss')
    tr_plt.set_xlabel('Epoch')
    tr_plt.legend(['loss', 'val_loss'], loc='upper left')

      
    predict_v = model.predict(testX)
    print('predict_v len pre-stack: ',len(predict_v))
    predict_ve = np.vstack(predict_v)
    predict_vec = np.array(np.vstack(predict_ve))
    print('predict_v len: ', predict_vec.shape)
    #predict_vec = predict_vec[0][0]
    print('predict_vec len: ',len(predict_vec))
    confidences = np.max(predict_vec, axis=1)
    print('testY len pre-stack: ',len(testY))
    testY1 = np.vstack(testY)
    print('testY len: ',testY1.shape)
    testYY = np.concatenate(testY1)
    print('testYY len: ',testYY.shape)
    
    predict_class = np.argmax(predict_vec, axis=1)
    predict = [lb.classes_[x] for x in predict_class]
    
    testX1 = np.vstack(testX)
    testXX = np.concatenate(testX1)
    print('testXX len: ',testXX.shape)

    #print("len(testYY)",len(testXX))
    testtimes = np.concatenate(TestTimes)
    print("len(TestTimes)",len(testtimes))
    
    for i in range(len(testYY)):
        #print(testYY[i])
        if testYY[i]==(len(target_names)-1):
            testYYY = testYY[:i]
            predict1 = predict[:i]
            testXXX = testXX[:i]
            ttimes = testtimes[:i]
            Confidences = confidences[:i]
            print(testYY[i])
            print(i)
            break
        else:
            testYYY = testYY
            predict1 = predict
            testXXX = testXX
            ttimes = testtimes
            Confidences = confidences

              
    print("len(testYYY)",len(testYYY))
    print("len(predict1)",len(predict1))
    
    result = classification_report(testYYY, predict1, labels=indices, target_names=target_names)
    print(result)
    cnf_matrix = confusion_matrix(testYYY, predict1)
    plot_confusion_matrix(cnf_matrix, fig, cm_plt, classes=target_names)
    fig.tight_layout()
    fig.show()
    
    Sloss = history.history['loss']

    return predict1, testYYY, testXXX, Confidences, ttimes, Sloss

def ApplyUnet(trainX, trainY, testX, testY,TestTimes, activities, nb_epoch=100, fi=32):
    
    target_names,indices = get_activity_index(activities)
    #print(target_names)
    indices.append(len(target_names))
    target_names.append('EPadding')
    
    trainY_b = np_utils.to_categorical(trainY, len(target_names))
    testY_b = np_utils.to_categorical(testY, len(target_names))
    lb = LabelBinarizer()
    
    print(indices)
    lb.fit(indices)
    
    model = build_unet_model(trainX.shape[2],trainX.shape[3],len(target_names),f=fi)
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["acc"])
    model.summary()
    
    model_checkpoint = keras.callbacks.ModelCheckpoint('bioattunet-2022_membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)
    history = model.fit(trainX, trainY_b, epochs=nb_epoch, validation_split=0.2,callbacks=[model_checkpoint], verbose=0)
    
    fig, (tr_plt,cm_plt) = plt.subplots(nrows=2, ncols=1, figsize=(6, 6))
    
    tr_plt.plot(history.history['loss'])
    tr_plt.plot(history.history['val_loss'])
    tr_plt.set_title('Model loss')
    tr_plt.set_ylabel('Loss')
    tr_plt.set_xlabel('Epoch')
    tr_plt.legend(['loss', 'val_loss'], loc='upper left')

      
    predict_v = model.predict(testX)
    print(predict_v)
    predict_vec = predict_v[0][0]
    confidences = np.max(predict_vec, axis=1)
    testYY = testY[0][0]
    
    predict_class = np.argmax(predict_vec, axis=1)
    predict = [lb.classes_[x] for x in predict_class]
    
    print("len(testYY)",len(testYY))
    print("len(predict)",len(predict))
    
    testXX = testX[0][0]
    
    for i in range(len(testYY)):
        #print(testYY[i])
        if testYY[i]==(len(target_names)-1):
            testYYY = testYY[:i]
            predict1 = predict[:i]
            testXXX = testXX[:i]
            ttimes = TestTimes[:i]
            Confidences = confidences[:i]
            print(testYY[i])
            print(i)
            break
        else:
            testYYY = testYY
            predict1 = predict
            testXXX = testXX
            ttimes = TestTimes
            Confidences = confidences

              
    print("len(testYYY)",len(testYYY))
    print("len(predict1)",len(predict1))
    
    result = classification_report(testYYY, predict1, labels=indices, target_names=target_names)
    print(result)
    testXX=testX
    cnf_matrix = confusion_matrix(testYYY, predict1)
    plot_confusion_matrix(cnf_matrix, fig, cm_plt, classes=target_names)
    fig.tight_layout()
    fig.show()
    
    Sloss = history.history['loss']

    return predict1, testYYY, testXXX, Confidences, ttimes, Sloss

# Run leave_one_bird_out Cross Validation

In [7]:
def leave_one_bird_out_cv(data_df,save_folder,simclass_d, nb_epoch=200, fi=10):

    data_df.drop(['year'],axis=1,inplace=True)
    data_df = data_df[data_df['label']!='unlabeled']
    data_df = data_df[data_df['label']!='unknown']
    birds = list(data_df.drop_duplicates(subset=['animal_tag'],keep = 'first')['animal_tag'])

    for bird in birds:
        #trainX,testX,trainY,testY,testtimes,selector = get_train_test_bird(data_df,trn=bird)
        trainX,testX,trainY,testY,testtimes,selector = get_train_test_bird_dev(data_df, trn=bird, reqlen=240, shuffle=False)
        trainX , testX = timeseries_standardize(trainX, testX)
        predict1, testYYY, testXXX, Confidences, Ttimes, Sloss = ApplyUnet_dev(trainX, trainY, testX, testY, testtimes, simclass_d, nb_epoch=nb_epoch, fi=fi)
        test_data_with_conf_pred = pd.DataFrame(data = {'timestamp':Ttimes, 'l_val':testYYY, 'predicted_label':predict1,'confidence':Confidences})
        save_fn = 'res_'+bird+'.csv'
        save_path = os.path.join(save_folder,save_fn)
        test_data_with_conf_pred.to_csv(save_path,index=False)

## Load Year data

In [8]:
data_df_O_2018 = pd.read_csv(r"G:\JaimeMorales\Codes\omizunagidori\database\omizunagidori\2018\l_2018_acc.csv")
data_df_O_2019 = pd.read_csv(r"G:\JaimeMorales\Codes\omizunagidori\database\omizunagidori\2019\l_2019_acc.csv")
data_df_O_2020 = pd.read_csv(r"G:\JaimeMorales\Codes\omizunagidori\database\omizunagidori\2020\l_2020_acc.csv")
data_df_O_2021 = pd.read_csv(r"G:\JaimeMorales\Codes\omizunagidori\database\omizunagidori\2021\l_2021_acc.csv")
data_df_O_2022 = pd.read_csv(r"G:\JaimeMorales\Codes\omizunagidori\database\omizunagidori\2022\l_2022_acc.csv")

In [9]:
data_df_U_2018 = pd.read_csv(r"G:\JaimeMorales\Codes\omizunagidori\database\umineko\2018\l_2018_acc.csv")
data_df_U_2019 = pd.read_csv(r"G:\JaimeMorales\Codes\omizunagidori\database\umineko\2019\l_2019_acc.csv")
data_df_U_2022 = pd.read_csv(r"G:\JaimeMorales\Codes\omizunagidori\database\umineko\2022\l_2022_acc.csv")

In [10]:
#data_df = pd.read_csv(r"G:\JaimeMorales\Codes\omizunagidori\database\omizunagidori\2018\l_2018_acc.csv")

In [11]:
#data_df = pd.concat([data_df_O_2018, data_df_O_2019, data_df_O_2020, data_df_O_2021, data_df_O_2022])
data_df = pd.concat([data_df_U_2018, data_df_U_2019, data_df_U_2022])

In [12]:
data_df

,animal_tag,timestamp,acc_x,acc_y,acc_z,label,l_val,year
0,LB07,1527210125000,0.811523,-0.028564,-0.071533,unlabeled,0,2018
1,LB07,1527210125040,0.863281,-0.050781,-0.047607,unlabeled,0,2018
2,LB07,1527210125080,0.831055,0.006104,0.024658,unlabeled,0,2018
3,LB07,1527210125120,0.839844,-0.002197,0.063477,unlabeled,0,2018
4,LB07,1527210125160,0.839355,-0.025879,0.077881,unlabeled,0,2018
...,...,...,...,...,...,...,...,...
4057058,LB09,1652788678838,-0.250000,-0.230469,0.937500,unlabeled,0,2022
4057059,LB09,1652788678870,-0.242188,-0.234375,0.925781,unlabeled,0,2022
4057060,LB09,1652788678903,-0.250000,-0.230469,0.929688,unlabeled,0,2022
4057061,LB09,1652788678935,-0.242188,-0.230469,0.929688,unlabeled,0,2022


## Load labels

In [13]:
#simclass_df = pd.read_csv(r'G:\JaimeMorales\Codes\omizunagidori\database\labels\O_labels_df.csv')
simclass_df = pd.read_csv(r'G:\JaimeMorales\Codes\omizunagidori\database\labels\U_labels_df.csv')
simclass_d = simclass_df['label'].to_dict()

## Get results

In [14]:
save_folder = r"G:\JaimeMorales\Codes\omizunagidori\database\results\baselines\sec10_att-Unet\all_umi"

In [15]:
leave_one_bird_out_cv(data_df,save_folder,simclass_d, nb_epoch=100, fi=12)

features:  Index(['acc_x', 'acc_y', 'acc_z'], dtype='object')
LB07
LB09
LB10
LB12
LB13
LB14
LB15
LB16
LB17
2019LB01
2019LB02
2019LB03
2019LB04
2019LB05
2019LB06
2019LB07
2019LB08
2019LB09
2019LB10
2019LB11
2019LB12
2019LB13
2019LB14
2019LBS01
2019LBS02
2019LBS03
LB02
LB03
LB08
LB07
train X(shape):  (4092, 1, 240, 3)
train Y(shape):  (4092, 1, 240)
test X(shape):  (147, 1, 240, 3)
test Y(shape):  (147, 1, 240)
times(shape):  (147, 240)
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1, 240, 3)]  0           []                               
                                                                                                  
 down_1_1 (Conv2D)              (None, 1, 240, 12)   120         ['input_1[0][0

<IPython.core.display.Javascript object>

5/5 [==============================] - 0s 7ms/step
predict_v len pre-stack:  147
predict_v len:  (35280, 20)
predict_vec len:  35280
testY len pre-stack:  147
testY len:  (147, 240)
testYY len:  (35280,)
testXX len:  (35280, 3)
len(TestTimes) 35280
len(testYYY) 35280
len(predict1) 35280
                      precision    recall  f1-score   support

           unlabeled       0.00      0.00      0.00         0
   ground_stationary       0.72      0.36      0.48     26095
             unknown       0.00      0.00      0.00         0
       flying_active       0.99      0.91      0.95      2525
      flying_passive       0.36      0.96      0.52      1044
          stationary       0.15      0.14      0.14      5579
        bathing_poss       0.00      0.00      0.00         0
             bathing       0.00      0.00      0.00         0
            preening       0.00      0.00      0.00         0
foraging_insect_poss       0.00      0.00      0.00         0
       ground_active       0.

<IPython.core.display.Javascript object>

12/12 [==============================] - 1s 32ms/step
predict_v len pre-stack:  358
predict_v len:  (85920, 20)
predict_vec len:  85920
testY len pre-stack:  358
testY len:  (358, 240)
testYY len:  (85920,)
testXX len:  (85920, 3)
len(TestTimes) 85920
len(testYYY) 85920
len(predict1) 85920
                      precision    recall  f1-score   support

           unlabeled       0.00      0.00      0.00         0
   ground_stationary       0.22      0.48      0.30       725
             unknown       0.00      0.00      0.00         0
       flying_active       0.97      0.97      0.97     55553
      flying_passive       0.53      0.42      0.47     11656
          stationary       0.19      0.12      0.15      9128
        bathing_poss       0.33      0.00      0.00      3732
             bathing       0.26      0.64      0.37      4865
            preening       0.00      0.00      0.00         0
foraging_insect_poss       0.00      0.00      0.00         0
       ground_active      

...

13/13 [==============================] - 0s 6ms/step
predict_v len pre-stack:  412
predict_v len:  (98880, 20)
predict_vec len:  98880
testY len pre-stack:  412
testY len:  (412, 240)
testYY len:  (98880,)
testXX len:  (98880, 3)
len(TestTimes) 98880
len(testYYY) 98880
len(predict1) 98880
                      precision    recall  f1-score   support

           unlabeled       0.00      0.00      0.00         0
   ground_stationary       0.00      0.00      0.00         0
             unknown       0.00      0.00      0.00         0
       flying_active       1.00      0.88      0.93     90354
      flying_passive       0.44      0.16      0.23      2407
          stationary       0.37      0.17      0.23      5659
        bathing_poss       0.00      0.00      0.00         0
             bathing       0.00      0.00      0.00       139
            preening       0.00      0.00      0.00         0
foraging_insect_poss       0.00      0.00      0.00         0
       ground_active       

<IPython.core.display.Javascript object>

7/7 [==============================] - 0s 6ms/step
predict_v len pre-stack:  213
predict_v len:  (51120, 20)
predict_vec len:  51120
testY len pre-stack:  213
testY len:  (213, 240)
testYY len:  (51120,)
testXX len:  (51120, 3)
len(TestTimes) 51120
len(testYYY) 51120
len(predict1) 51120
                      precision    recall  f1-score   support

           unlabeled       0.00      0.00      0.00         0
   ground_stationary       0.00      0.06      0.00       201
             unknown       0.00      0.00      0.00         0
       flying_active       0.96      0.96      0.96     34305
      flying_passive       0.21      0.52      0.29      1671
          stationary       0.74      0.18      0.30     10458
        bathing_poss       0.00      0.00      0.00         0
             bathing       0.67      0.45      0.54      4128
            preening       0.00      0.00      0.00       108
foraging_insect_poss       0.00      0.00      0.00         0
       ground_active       0.

<IPython.core.display.Javascript object>

9/9 [==============================] - 0s 7ms/step
predict_v len pre-stack:  283
predict_v len:  (67920, 20)
predict_vec len:  67920
testY len pre-stack:  283
testY len:  (283, 240)
testYY len:  (67920,)
testXX len:  (67920, 3)
len(TestTimes) 67920
len(testYYY) 67920
len(predict1) 67920
                      precision    recall  f1-score   support

           unlabeled       0.00      0.00      0.00         0
   ground_stationary       0.73      0.11      0.19     20280
             unknown       0.00      0.00      0.00         0
       flying_active       0.97      0.91      0.94     31170
      flying_passive       0.36      0.36      0.36      6672
          stationary       0.03      0.68      0.06       942
        bathing_poss       0.00      0.00      0.00         0
             bathing       0.33      0.33      0.33      8182
            preening       0.00      0.00      0.00       316
foraging_insect_poss       0.00      0.00      0.00         0
       ground_active       0.

# Run induvidual test

## Load Data

In [16]:
data_df = pd.read_csv(r"G:\JaimeMorales\Codes\omizunagidori\database\omizunagidori\2022\l_2022_acc.csv")
data_df

,animal_tag,timestamp,acc_x,acc_y,acc_z,label,l_val,year
0,LB01,1661544781000,0.011719,-0.105469,0.273438,unlabeled,0,2022
1,LB01,1661544781032,0.105469,-0.199219,0.574219,unlabeled,0,2022
2,LB01,1661544781064,-0.183594,0.175781,0.066406,unlabeled,0,2022
3,LB01,1661544781096,-0.550781,-0.027344,-0.144531,unlabeled,0,2022
4,LB01,1661544781129,-0.363281,-0.132813,-0.035156,unlabeled,0,2022
...,...,...,...,...,...,...,...,...
22942599,LB17,1663492499838,-0.062500,-0.234375,2.093750,unlabeled,0,2022
22942600,LB17,1663492499870,-0.433594,-0.074219,2.031250,unlabeled,0,2022
22942601,LB17,1663492499903,-0.304688,-0.320313,1.878906,unlabeled,0,2022
22942602,LB17,1663492499935,-0.062500,-0.554688,1.621094,unlabeled,0,2022


In [17]:
simclass_df = pd.read_csv(r'G:\JaimeMorales\Codes\omizunagidori\database\labels\O_labels_df.csv')
simclass_d = simclass_df['label'].to_dict()
#simclass_d[len('simclass_d')]='pad_class'
simclass_d

{0: 'unlabeled',
 1: 'stationary',
 2: 'flight_take_off',
 3: 'flight_cruising',
 4: 'unknown',
 5: 'bathing',
 6: 'surface_seizing',
 7: 'preening',
 8: 'foraging_dive',
 9: 'body_shaking'}

In [18]:
data_df.drop(['year'],axis=1,inplace=True)
data_df = data_df[data_df['label']!='unlabeled']

In [19]:
birds = list(data_df.drop_duplicates(subset=['animal_tag'],keep = 'first')['animal_tag'])
birds

['LB01',
 'LB02',
 'LB03',
 'LB04',
 'LB05',
 'LB08',
 'LB09',
 'LB11',
 'LB13',
 'LB14',
 'LB15',
 'LB17']

## Divide train Test

In [20]:
#trainX,testX,trainY,testY,testtimes,selector = get_train_test_bird(data_df,trn='LB0x')
trainX,testX,trainY,testY,testtimes,selector = get_train_test_bird_dev(data_df, shuffle=False, trn='LB0x', reqlen=10000)

features:  Index(['acc_x', 'acc_y', 'acc_z'], dtype='object')
LB01
LB02
LB03
LB04
LB05
LB08
LB09
LB11
LB13
LB14
LB15
LB17
LB0x


ValueError: list.remove(x): x not in list

In [ ]:
print(selector)

In [ ]:
test_data = data_df[data_df['animal_tag']==selector]
test_data

In [ ]:
print('trainX(shape): ',trainX.shape)
print('trainY(shape): ',trainY.shape)

print('testX(shape): ',testX.shape)
print('testY(shape): ',testY.shape)
print('testtimes(len): ',len(testtimes))

In [ ]:
trainX , testX = timeseries_standardize(trainX, testX)

## Network testing

In [ ]:
#predict1, testYYY, testXXX, Confidences, Ttimes, Sloss = ApplyUnet(trainX, trainY, testX, testY, testtimes, simclass_d, nb_epoch=100, fi=10)
predict1, testYYY, testXXX, Confidences, Ttimes, Sloss = ApplyUnet_dev(trainX, trainY, testX, testY, testtimes, simclass_d, nb_epoch=100, fi=10)

In [ ]:
len(Ttimes)

In [ ]:
test_data_with_conf_pred = pd.DataFrame(data = {'time':Ttimes[-1], 'l_val':testYYY, 'predicted_label':predict1,'confidence':Confidences})

In [ ]:
save_folder = r'G:\JaimeMorales\Codes\omizunagidori\database\results\baselines'
save_fn = 'res_'+selector+'.csv'
save_path = os.path.join(save_folder,save_fn)
save_path

In [ ]:
test_data_with_conf_pred.to_csv(save_path,index=False)

In [ ]:
tttimes = range(len(predict1))

In [ ]:
labels,indices = get_activity_index(simclass_d)
indices.append(11)
labels.append('EPadding')
figs = plt.figure('result',figsize=(10, 4))
axs = plt.subplot(111)
axs.plot(tttimes, predict1, color='r')
axs.plot(tttimes, testYYY, color='g')
axs.set_ylabel('Activity label')
axs.set_yticks(indices)
axs.set_yticklabels(labels)
figs.show()